In [62]:
from selenium import webdriver
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from datetime import date, datetime
from selenium.common.exceptions import TimeoutException
import time

In [155]:
driver = webdriver.Chrome()
page = driver.get("https://www.adapt.io/directory/industry/telecommunications/A-1")


In [122]:
links_of_alphabets = []
alphabets = driver.find_elements_by_class_name("DirectoryTopInfo_linkItemWrapper__2MyQQ")

# collect links (href) alphabetically(A, B, .... Z)
for link in alphabets:
    links_of_alphabets.append(link.find_elements_by_tag_name('a')[0].get_attribute('href'))

In [128]:
# print first 5 link text
for link in links_of_alphabets[:5]:
    print(link)

https://www.adapt.io/directory/industry/telecommunications/A-1
https://www.adapt.io/directory/industry/telecommunications/B-1
https://www.adapt.io/directory/industry/telecommunications/C-1
https://www.adapt.io/directory/industry/telecommunications/D-1
https://www.adapt.io/directory/industry/telecommunications/E-1


In [129]:
# this function collects the source name and source url for each company listed on the page

def get_company_index_data_of_single_page():
    number_of_attempts = 0
    should_try = 5
    list_of_companies_in_single_page = []
    
    while (number_of_attempts <= should_try):
        try:
            time.sleep(3)
            companies = driver.find_elements_by_class_name("DirectoryList_linkItemWrapper__3F2UE")

            for company in companies:
                company_dict = {
                    "company_name":company.text,
                    "source_url":company.find_elements_by_tag_name('a')[0].get_attribute('href')
                }
                # append the dictionary into a list
                list_of_companies_in_single_page.append(company_dict)

            if len(companies) == len(list_of_companies_in_single_page):
                for company in list_of_companies_in_single_page:
                    list_of_companies.append(company)
                break
            else:
                print("could not collect info of {} companies present in the page: ", len(companies))
                number_of_attempts += 1
                continue
                
        except Exception as exc:
            print(exc)
            

In [145]:
# this function checks if a link named Next is present in the page or not.

def click_next_button():
    attempt = 0
    while(attempt < 4):
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            next_button = driver.find_element_by_link_text('Next')
            
            # If the next button is found, then click on the Next link
            if next_button:
                next_button.click()
                return "continue"
            else:
                raise Exception
        except Exception as exc:
            attempt += 1
            time.sleep(2)
        
    return "break"

In [149]:
list_of_companies = []
number_of_companies_present_in_single_category = 0


# start getting each link alphabetically
for link in links_of_alphabets:
    driver = webdriver.Chrome()
    page = driver.get(link)
    
    while(1):
        number_of_companies = get_company_index_data_of_single_page()

        # call click_next_button function to look for the link to go to the Next page.
        # If found, then click on it and start collecting information.
        # otherwise, close the window and go to next URL.
        state = click_next_button()
        if state == "continue":
            continue
        else:
            break
            
    print("****    got info of {} companies from {}   ****\n".format(len(list_of_companies) - number_of_companies_present_in_single_category, link[-3:]))
    number_of_companies_present_in_single_category = len(list_of_companies)
    driver.close()

    
print("finally, got info of total {} companies".format(len(list_of_companies)))


****    got info of 412 companies from A-1   ****

****    got info of 175 companies from B-1   ****

****    got info of 443 companies from C-1   ****

****    got info of 145 companies from D-1   ****

****    got info of 160 companies from E-1   ****

****    got info of 130 companies from F-1   ****

****    got info of 166 companies from G-1   ****

****    got info of 89 companies from H-1   ****

****    got info of 172 companies from I-1   ****

****    got info of 33 companies from J-1   ****

****    got info of 65 companies from K-1   ****

****    got info of 125 companies from L-1   ****

****    got info of 271 companies from M-1   ****

****    got info of 191 companies from N-1   ****

****    got info of 112 companies from O-1   ****

****    got info of 222 companies from P-1   ****

****    got info of 23 companies from Q-1   ****

****    got info of 144 companies from R-1   ****

****    got info of 395 companies from S-1   ****

****    got info of 640 companies f

In [154]:
# print first 5 companies

list_of_companies[:5]

[{'company_name': 'A + Communications and Security',
  'source_url': 'https://www.adapt.io/company/a--communications-and-security'},
 {'company_name': 'A&A Technology Group',
  'source_url': 'https://www.adapt.io/company/a-a-technology-group'},
 {'company_name': 'A Better Answer',
  'source_url': 'https://www.adapt.io/company/a-better-answer-4'},
 {'company_name': 'A Cheerful Giver',
  'source_url': 'https://www.adapt.io/company/a-cheerful-giver-inc-1'},
 {'company_name': 'A-CTI',
  'source_url': 'https://www.adapt.io/company/a-cti-1'}]

In [152]:
import json

# function to add to JSON 
def write_json(data, filename='company_index.json'): 
    with open(filename,'w') as f: 
        json.dump(data, f, indent=4) 
      
write_json(list_of_companies) 
# print(all_companies)